# Tutorial 2: Event-based Star Data

Prepared by the International Centre for Neuromorphic Systems (ICNS) at Western Sydney University, adapted from the short course presented at the AMOS Conference 2023.

## Introduction 

***What you will learn***
In this tutorial, we will work with real-world event-based data to learn how to load and process it. 

By the end of this tutorial, you will...
* Be able to load and view event-based star data
* Ignore and remove hot pixels
* Visualise the event-based data as an image
* Calculate the pixel rate of sideral motion


In [ ]:
# Install the required dependencies (if not already installed). This is required to install the required packages in Google Colaboratory.
!pip3 install event_stream tonic

### Downloading the data

If you are running this tutorial on Google Colab, you will need to download the required data file from Github. The command below will automatically download the correct file and place it in the same directory as this tutorial. If you are running the tutorial locally, you may already have this file locally and do not need to download it again. The data file can be found in the Github repository at https://github.com/neuromorphicsystems/tutorials/raw/main/data/stars.es.

Note that this data file is approximately 55 MB in size. It may take a few minutes to download.

In [ ]:
import urllib.request


# Download the sample data from Github (55 MB)
stars_path, _ = urllib.request.urlretrieve(
    "https://github.com/neuromorphicsystems/tutorials/raw/main/data/stars.es",
    "stars.es",
)

### Loading the data
Now that we have some data locally, we can now load the event-based data from the file. For this example, we will load the entire file into memory for speed and convenience. We're going to add an extra field title 'p' to the last column, because transformations in Tonic are looking for it. We'll also flip the image along the vertical axis.

In [ ]:
import event_stream
import numpy


decoder = event_stream.Decoder(stars_path)
events = numpy.concatenate([chunk for chunk in decoder])
events = events.view(dtype=events.dtype.descr[:3] + [(('on', 'p'), '?')])
events['y'] = decoder.height - 1 - events['y']

### Viewing the data as an accumulated frame

A very useful way to visualise event-based star data is to accumulate all the events in the file together to generate a single frame that captures all the data within the file. For that we'll be using Tonic's [ToImage](https://tonic.readthedocs.io/en/latest/auto_examples/representations/plot_toimage.html#sphx-glr-auto-examples-representations-plot-toimage-py) transform. This is a very similar method that we used in the previous tutorial to render frames, except that we count the number of events at each pixel instead of just marking whether an event occurred or not. We also apply it across the whole recording.

This technique works particularly well on star data as it is inherently a sparse dataset. If you were to do the same to the recordings in the previous tutorial, the results would be very hard to interpret. As before, it is important to ensure that the visualisations used are appropriate to the dataset.

We can use the code below to calculate an accumulated frame from the event-based data:

In [ ]:
from tonic import transforms

# create the transformation that we'll be able to re-use
image_transform = transforms.ToImage(sensor_size=(decoder.width, decoder.height, 2))

# transform the events into a single image
accumulated_frame = image_transform(events)

# sum over ON and OFF channels
accumulated_frame = accumulated_frame.sum(0)

We can then plot this data using Matplotlib as follows:

In [ ]:
import matplotlib
import matplotlib.colors
import matplotlib.pyplot
import matplotlib.transforms

matplotlib.rcParams["figure.figsize"] = [12, 8]

matplotlib.pyplot.imshow(
    accumulated_frame,
    cmap="magma",
)

And... we get what looks like a blank image. What's happening here?

The problem is one of scale. Let's take a look at a histogram of the number of events per pixel in the above image. We can do that using the following code:

In [ ]:
matplotlib.pyplot.hist(accumulated_frame.ravel(), bins=256)
matplotlib.pyplot.yscale("log")
matplotlib.pyplot.show()

Clearly, the event rate is dominated by just a handful of pixels that are generating the majority of the events. These are called Hot Pixels. They are more sensitive than their neighbors due to transistor mismatch.

If we plot the log of the accumulated value, we can better visualise the data. We can use the following code to better view the starfield:

In [ ]:
import matplotlib
import matplotlib.colors
import matplotlib.pyplot
import matplotlib.transforms

matplotlib.rcParams["figure.figsize"] = [12, 8]
matplotlib.pyplot.imshow(
    numpy.log(accumulated_frame + 1),
    cmap="magma",
)

However, there is another hidden aspect to the data that isn't show by the accumulated frame. If you watch the rendered video of the data, you will notice that there is a flash of events at the start of the recording. This is due to the camera powering up and adjusting to the scene dynamics. If you look at a plot of the event rate, you can clearly see the impact of this burst of events:

In [ ]:
%matplotlib inline

import event_stream
import matplotlib.pyplot

decoder = event_stream.Decoder(stars_path)

matplotlib.pyplot.plot(events["t"])
matplotlib.pyplot.xlabel("Event Index")
matplotlib.pyplot.ylabel("Time (us)")
matplotlib.pyplot.title("Plot of Timestamps vs. Index")
matplotlib.pyplot.show()

The timestamps grow very slowly over time for the first 6,000,000 events ($6e6$ events) and then start increasing more rapidly after that point. That indicates that a significant number of events were generated over a relatively short period of time. This data isn't terribly useful to us at present, so we can remove that data by simply plotting the events occurring after that index. 

We can use the code below to do that:

In [ ]:
import matplotlib.pyplot
import matplotlib.colors

accumulated_frame = image_transform(events[6000000:]).sum(0)

matplotlib.rcParams["figure.figsize"] = [12, 8]
matplotlib.pyplot.imshow(
    accumulated_frame,
    norm=matplotlib.colors.LogNorm(vmax=numpy.percentile(accumulated_frame, 99.9)),
    cmap="magma",
)

The plotted data does not look significantly different, despite us dropping about 6 million events from the data. For convenience sake, we have already removed this noisy data and saved the result as https://github.com/neuromorphicsystems/tutorials/raw/main/data/stars_cut.es.

We will use this shortened version for the data in the next tutorial (3. Star mapping, https://colab.research.google.com/github/neuromorphicsystems/tutorials/blob/main/3.%20Star%20mapping.ipynb).

### Calculating the sidereal rate for the data

Let's now try to calculate the sidereal rate of movement of the stars in pixel coordinates. We will need this value in order to perform the event-warping needed to extract star positions from this data. 

The simplest means to calculate this value is to measure it by hand. To do this, we will extract a short extract of the recording. We will start by extracting about 4,000,000 events. Given that the sidereal rate is relatively constant, we could pick any length recording to do so.

We can then plot the extracted section as follows:

In [ ]:
accumulated_frame = image_transform(events[6000000:10000000]).sum(0)

matplotlib.rcParams["figure.figsize"] = [12, 8]
matplotlib.pyplot.imshow(
    accumulated_frame,
    norm=matplotlib.colors.LogNorm(vmax=numpy.percentile(accumulated_frame, 99.9)),
    cmap="magma",
)

Now, let's pick a streak and extract a small region around it. We will pick the streak located around $(350, 800)$ in the image. We can extract and plot this portion of our extracted recording using the following:

In [ ]:
matplotlib.rcParams["figure.figsize"] = [12, 8]

matplotlib.pyplot.imshow(
    accumulated_frame[340:400, 765:900],
    cmap="magma",
)

start_point = [6, 10]
end_point = [128, 46]

matplotlib.pyplot.scatter(
    x=[start_point[0], end_point[0]],
    y=[start_point[1], end_point[1]],
    marker="o",
    facecolor="none",
    edgecolors="#ffffff",
    linewidth=0.4,
    s=100,
)

We can then easily calculate the sidereal motion by finding the starting and end points of the streak and dividing by the elapsed time. Keep in mind that we used the indices to extract a portion of the recording, and not the actual time, therefore we need to extract the time from the event at the index corresponding to the start and end of our extracted range.

We can then calculate the pixel speed of the sidereal motion as follows:

In [ ]:
start = [events[6000000]["t"], 6, 9]
end = [events[10000000]["t"], 128, 46]

dt = events[10000000]["t"] - events[6000000]["t"]
dx = end_point[0] - start_point[0]
dy = end_point[1] - start_point[1]
print(f"Δt: {dt} µs")
print(f"Δx/Δt: {(dx / dt) * 1e6:.3f} pixels per second")
print(f"Δy/Δt: {(dy / dt) * 1e6:.3f} pixels per second")

We will use this in the next tutorial to perform the star mapping.